In [130]:
from ngram import BasicNgram
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
word_detokenize = TreebankWordDetokenizer().detokenize

START_SYMBOL="<$>"
END_SYMBOL="</$>"

def corpus(corpus):
    with open(corpus, 'r') as f:
        data = [word_tokenize(sent) for sent in sent_tokenize(f.read())]
        out = [START_SYMBOL] + data[0]
        for sent in data[1:]:
            out.append(END_SYMBOL)
            out.append(START_SYMBOL)
            out += sent
        out.append(END_SYMBOL)
    return out

def create_story(corpusname, nlength=2, wordlength=100):
    data = corpus(corpusname)
    ngram = BasicNgram(nlength, data, start_symbol=START_SYMBOL, end_symbol=END_SYMBOL)
    context = [ngram._start_symbol]*(nlength-1)
    output = []
    for _ in range(wordlength):
        word = ngram[tuple(context)].generate()
        context.append(word)
        context.pop(0)
        output.append(word)
    output = [x for x in output if x not in {START_SYMBOL, END_SYMBOL}]
    print(word_detokenize(output).replace(' .', '.'))


ImportError: cannot import name 'BasicNgram' from 'ngram' (/home/vilda/uds-student/computational_linguistics/hw1/ngram.py)

In [116]:
create_story('../data/Junglebook.txt', nlength=2)

There was weak spot, here is wilder work. I've been halibut along my way with men did not a corner and only three elephants broke, and again , 1894, Kotick followed him here a fraction of being noticed by in twenty years, wicked tails cold sound, but all out one another pack a clear of the grass quivered again. Now we can ,"_Maro!"`` Umph! he is a noise of``So bounding all seem to safely


In [117]:
create_story('../data/King James Bible.txt', nlength=4)

In the year that king Uzziah died I saw also the LORD hath brought me in to possess the iniquities of our fathers. Nathanael saith unto him, Is there not here a prophet of the LORD thy God with all thine offerings thou shalt offer the third part of you that is sorry for me, and I laid meat unto them. A wise king scattereth the wicked, and plucked the spear out of the captivity, and have made a man every whit whole on the sabbath it shall be ,


https://www.kaggle.com/ultrajack/modern-renaissance-poetry  

https://www.kaggle.com/tgdivy/poetry-foundation-poems

In [126]:
create_story('../data/poetryfoundation_clean.txt', nlength=4, wordlength=30)

Poem Objects Used to Prop Open a Window because this window is split. Rip into. * A poem should be motionless in time
